In [1]:
import os
import re
import pandas as pd
import spacy
import en_core_web_sm
# from spacy import doc
import csv
from pathlib import Path
from pathlib import PureWindowsPath

In [2]:
def extract_chunks(input_text):
    doc = nlp(input_text)
    spans = list(doc.ents)  #+ list(doc.noun_chunks)
    out_list = []
    for span in spans:
        span.merge()
        out_list.append(span)
    return out_list


In [3]:
def extract_name_pair(name, input_text):
    out_list = []
    if not pd.isnull(input_text):
        doc = nlp(input_text)
        spans = list(doc.ents)  #+ list(doc.noun_chunks)

        for item in spans:
            try:
                item = re.sub(pattern='[^a-zA-Z0-9 -]',repl='',string=item.string)            
                if (re.search(item,name,flags=re.IGNORECASE)):
                    out_list.append(item)

            except:
                pass
            
    return out_list
#     else:
#         return None

In [4]:
def basic_wc(input_text):
    if not (pd.isnull(input_text)):
        sent = re.sub(pattern='[^a-zA-Z0-9 -]',repl='',string=input_text)            
        doc = nlp(sent)
        return doc.__len__()
    else:
        return 0

In [41]:
def split_quarter(input_text):
#     print(input_text)
    try:                
#         print(pd.isnull(input_text))
        if not (pd.isnull(input_text)):
            input_text = re.search('^.*\((q[1-4]),.*', input_text, re.IGNORECASE)
#             print(input_text,'##')
            return input_text.group(1).strip()
        return None
    except:
        return None
def split_year(input_text):
    try:                
        if not (pd.isnull(input_text)):
            input_text = re.search(', ([0-9]+)\)', input_text, re.IGNORECASE)
            return input_text.group(1).strip()
        return None
    except:
        return None    
    


In [6]:
nlp = en_core_web_sm.load()

In [23]:
# collection = "html"
# data_folder = Path(collection)
output_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/analyst')
meta_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/meta')
input_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/extract')

n_test=10
file_list = os.listdir(input_folder)
out_list =[]
# for i in range(len(file_list)):
print('FILE\tANALYST_Flag\tFIRM\tQuarter\tYear') 
print(len(file_list))
# for i in range(n_test):
for i in range(len(file_list)):
    if(re.search('csv',file_list[i],flags=re.IGNORECASE)):
        out_list.append(file_list[i])
#             print(file_list[i])

out_pd = pd.DataFrame(out_list,columns=['file'])
out_pd.to_csv(meta_folder / 'analyst_file_list.csv',sep=',',encoding='utf-8',index_label='index')

FILE	ANALYST_Flag	FIRM	Quarter	Year
80853


In [34]:
file_csv = pd.read_csv(meta_folder / 'analyst_file_list.csv',index_col='index')

In [35]:
file_csv.at[0,'file']

'analyst_out_foringestion_10.csv'

In [46]:
import csv
fieldnames = ['index','fname', 'status']

# with open('/Users/lky/workspace/convoparse/textparsing/log_ner_analyst_rerun.csv', 'w') as csvfile:
with open(meta_folder / 'log_analyst_repeat.csv', 'w') as csvfile:
    logwriter = csv.writer(csvfile,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
#     logwriter.writerow(fieldnames)

    for ii in range(0,20000):
        try:
            # out_analyst= pd.read_csv('analyst_out_foringestion.csv',encoding='utf-8',na_values='NaN')
#             filename = '/Volumes/S/convoparse/extract/analyst_out_foringestion_' + str(ii)+'.csv' 
#             filename = '/Volumes/Transcend/extract/analyst_out_foringestion_' + str(ii)+'.csv' 
            filename = file_csv.at[ii,'file']

            #         extract/analyst_out_foringestion_'+str(ii)+'.csv' 
#             fileout = '/Users/lky/workspace/convoparse/textparsing/test/analyst_out_foringestion_'+str(ii)+'.csv' 
            fileout = 'withcount_' + file_csv.at[ii,'file']

#             out_analyst= pd.read_csv(filename,encoding='utf-8',na_values='NaN',header=None,\
            out_analyst= pd.read_csv(input_folder / filename,encoding='utf-8',na_values='NaN',header=None,\
                                     names=['index','file','quarter_year','executive_firm','ticker','analyst_name',\
                                           'analyst_firm','analyst_content','analyst_qanda','exec_name','exec_title'])
    #         print(out_analyst)
            out_analyst['quarter'] = out_analyst.apply(lambda x: \
                                                     split_quarter(x['quarter_year']), axis=1)

            out_analyst['year'] = out_analyst.apply(lambda x: \
                                                     split_year(x['quarter_year']), axis=1)

            out_analyst['exec_mention_qna'] = out_analyst.apply(lambda x: extract_name_pair(x['exec_name'],x['analyst_qanda']), axis=1)
            out_analyst['exec_mention_transcript'] = out_analyst.apply(lambda x: extract_name_pair(x['exec_name'],x['analyst_content']), axis=1)
            out_analyst['exec_count_qna'] = out_analyst.apply(lambda x: \
                                                     len(x['exec_mention_qna']), axis=1)

            out_analyst['exec_count_transcript'] = out_analyst.apply(lambda x: \
                                                     len(x['exec_mention_transcript']), axis=1)
            out_analyst['exec_count_total']= out_analyst.exec_count_qna+out_analyst.exec_count_transcript
            out_analyst['analyst_qna_count'] = out_analyst.apply(lambda x: \
                                                     basic_wc(x['analyst_qanda']), axis=1)
            out_analyst['analyst_content_count'] = out_analyst.apply(lambda x: \
                                                     basic_wc(x['analyst_content']), axis=1)
            out_analyst['analyst_total_wc'] = out_analyst['analyst_qna_count'] + out_analyst['analyst_content_count']

            # out_analyst[['file','quarter_year','executive_firm','ticker',\
            # 'analyst_name','analyst_firm','analyst_total_wc','exec_name','exec_title','exec_mention_qna','exec_count_qna',
            #             'exec_mention_transcript','exec_count_transcript','exec_count_total']]\
            # .to_csv('analyst_out_withcount.csv',sep=',', encoding = 'utf-8')
            # out_executives.select({[}'file','quarter_yer'])

            out_analyst[['file','quarter','year','executive_firm','ticker',\
            'analyst_name','analyst_firm','analyst_total_wc','exec_name','exec_title','exec_count_total']]\
            .to_csv(output_folder / fileout,sep=',', encoding = 'utf-8',header=False)
#             .to_csv(fileout,sep=',', encoding = 'utf-8',header=False)
        #     .to_csv('analyst_out_withcount.csv',sep=',', encoding = 'utf-8')    
            logwriter.writerow([ii,fileout ,'success'])
            print(ii,'##success')

        except:
            print(ii,'##fail')
            logwriter.writerow([ii,fileout ,'fail'])
            
            pass

0 ##success
1 ##success
2 ##success
3 ##success
4 ##success
